In [2]:
!pip install scikit-plot
# !pip uninstall -v scikit-learn
# !pip install -v scikit-learn

**Importing required libraries**

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline 
import matplotlib.pyplot as plt

**Load Data :**

Here we will load the dataset, which is collected by the researches manually. we will use this data set to train our model.

In [6]:
data = pd.read_csv("mypersonality_final.csv", encoding='cp1252')
data.head()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


**Analysis of Dataset**

here we will do some analysis on loaded dataset, so that we can understand dataset well like what type of data is there, type of the data, missing values, corelation between all the variables etc. 

**Drop un-necessary columns**

here we will remove some un-wanted columns

In [7]:
df = data[["STATUS","cEXT","cNEU","cAGR","cCON","cOPN"]]
df.tail(10)

,STATUS,cEXT,cNEU,cAGR,cCON,cOPN
9907,Oh well. Only two things to do when your team...,n,n,n,y,y
9908,And the cabin fever begins.,n,n,n,y,y
9909,Facebook me marea. Me hates it long time T-T,n,y,n,n,y
9910,Red,y,n,y,y,n
9911,"About mornings and winter,and magic.",n,n,y,n,y
9912,little things give you away.,n,n,y,n,y
9913,is wishing it was Saturday.,y,y,y,y,y
9914,is studying hard for the G.R.E.,y,y,y,y,y
9915,snipers get more head,n,y,n,n,y
9916,Last night was amazing! Not only did I see *PR...,y,y,n,y,y


In [8]:
#check for null data
df.isnull().sum()

STATUS    0
cEXT      0
cNEU      0
cAGR      0
cCON      0
cOPN      0
dtype: int64

***Data Distribusion chart***

**NLP Pre-Processing**

here we will do some nlp pre-processing on data like lower case conversion, drop null values, remove html tags, remove urls , remove numbers, characters, stop words, Contraction to Expansion, stemming etc.

In [9]:
#Reusable Code for NLP Preprocessing 
###########
# used for dataframe related stufs
###########
class CleanDataFrame():
    
    def dropna(self,input_df):
        #print("DropNA",input_df.isnull().values.any())
        if input_df.isnull().values.any():
            return input_df.dropna()
        else:
            return input_df
        
    def to_lower(self,input_df):
        #print("ToLower")
        input_df = input_df.apply(lambda x: x.astype(str).str.lower())
        return input_df
    
    def to_upper(self,input_df):
        input_df = input_df.apply(lambda x: x.astype(str).str.upper())
        return input_df  

    def remove_single_words(self,input_df):
        input_df = input_df[[len(i.split())>1 for i in input_df['STATUS'].astype(str).values]]
        return input_df
    

###########
# used for text related stufs
###########
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re
import string
import nltk
nltk.download('punkt')
nltk.download('wordnet')
# nltk.download("popular")  #this will be required onetime only
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

class TextPreProcessing():
    # make all text lowercase
    def text_lowercase(self,text):
        return text.lower()
    # remove HTML tags
    def remove_html_tags(self,text):
        html_pattern = r'<.*?>'
        without_html = re.sub(pattern=html_pattern, repl=' ', string=text)
        return without_html
    # remove url
    def remove_urls(self,text):
        new_text = re.sub(r'www\S+|http\S+', '', text)
        return new_text
    # remove numbers
    def remove_numbers(self,text):
        result = re.sub(r'\d+', '', text)
        return result
    
    # remove special char
    def remove_specialchar(self,text):
        result = re.sub('[^A-Za-z0-9]+', '', text)
        return result
    
    # Contraction to Expansion
    def contraction_to_expansion(self,text):
        contractions = { 
            "ain't": "am not",
            "aren't": "are not",
            "can't": "cannot",
            "can't've": "cannot have",
            "'cause": "because",
            "could've": "could have",
            "couldn't": "could not",
            "couldn't've": "could not have",
            "didn't": "did not",
            "doesn't": "does not",
            "don't": "do not",
            "hadn't": "had not",
            "hadn't've": "had not have",
            "hasn't": "has not",
            "haven't": "have not",
            "he'd": "he would",
            "he'd've": "he would have",
            "he'll": "he will",
            "he'll've": "he will have",
            "he's": "he is",
            "how'd": "how did",
            "how'd'y": "how do you",
            "how'll": "how will",
            "how's": "how does",
            "i'd": "i would",
            "i'd've": "i would have",
            "i'll": "i will",
            "i'll've": "i will have",
            "i'm": "i am",
            "i've": "i have",
            "isn't": "is not",
            "it'd": "it would",
            "it'd've": "it would have",
            "it'll": "it will",
            "it'll've": "it will have",
            "it's": "it is",
            "let's": "let us",
            "ma'am": "madam",
            "mayn't": "may not",
            "might've": "might have",
            "mightn't": "might not",
            "mightn't've": "might not have",
            "must've": "must have",
            "mustn't": "must not",
            "mustn't've": "must not have",
            "needn't": "need not",
            "needn't've": "need not have",
            "o'clock": "of the clock",
            "oughtn't": "ought not",
            "oughtn't've": "ought not have",
            "shan't": "shall not",
            "sha'n't": "shall not",
            "shan't've": "shall not have",
            "she'd": "she would",
            "she'd've": "she would have",
            "she'll": "she will",
            "she'll've": "she will have",
            "she's": "she is",
            "should've": "should have",
            "shouldn't": "should not",
            "shouldn't've": "should not have",
            "so've": "so have",
            "so's": "so is",
            "that'd": "that would",
            "that'd've": "that would have",
            "that's": "that is",
            "there'd": "there would",
            "there'd've": "there would have",
            "there's": "there is",
            "they'd": "they would",
            "they'd've": "they would have",
            "they'll": "they will",
            "they'll've": "they will have",
            "they're": "they are",
            "they've": "they have",
            "to've": "to have",
            "wasn't": "was not",
            " u ": " you ",
            " ur ": " your ",
            " n ": " and ",
            "won't": "would not",
            'dis': 'this',
            'bak': 'back',
            'brng': 'bring'}
        if type(text) is str:
            for key in contractions:
                value = contractions[key]
                text = text.replace(key,value)
            return text
        else:
            return text
    # remove punctuation - The following code removes this set of symbols [!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]:
    def remove_punctuation(self,text):
        return text.translate(str.maketrans('', '', string.punctuation))

    ## Remove single characters
    def remove_single_char(self,text):
        return re.sub(r"\b[a-zA-Z]\b", "", text)
    # tokenize
    def tokenize(self,text):
        text = word_tokenize(text)
        return text
    # remove stopwords
    def remove_stopwords(self,text): 
        return " ".join([t for t in text if t not in STOP_WORDS])
    # lemmatize string 
    def lemmatize_word(self,text): 
        #word_tokens = word_tokenize(text) 
        # provide context i.e. part-of-speech
        lemmatizer = WordNetLemmatizer()
        lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in text] 
        return lemmas 

    def preprocessing(self,text):
        #text = self.text_lowercase(text)
        text = self.remove_urls(text)
        text = self.remove_html_tags(text)
        text = self.contraction_to_expansion(text)
        text = self.remove_numbers(text)
        text = self.remove_punctuation(text)
        text = self.remove_single_char(text)
        text = self.tokenize(text)
        text = self.lemmatize_word(text)
        text = self.remove_stopwords(text)
        return text    

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
#clean data
objTextPreProcessing = TextPreProcessing()
df['STATUS'] =  df['STATUS'].apply(lambda x: objTextPreProcessing.preprocessing(x)) #apply nlp pre-processing on textcolumn

objCleanDataFrame = CleanDataFrame()
df = objCleanDataFrame.dropna(df)   #drop null values from dataframe
df = objCleanDataFrame.to_lower(df) #convert dataframe to lowercase
df = objCleanDataFrame.remove_single_words(df) #drop single word row from dataframe
df.head(10)

<ipython-input-10-64c017eae3c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STATUS'] =  df['STATUS'].apply(lambda x: objTextPreProcessing.preprocessing(x)) #apply nlp pre-processing on textcolumn


,STATUS,cEXT,cNEU,cAGR,cCON,cOPN
0,like sound thunder,n,y,n,n,y
1,sleepy funny sleep,n,y,n,n,y
2,sore want knot muscle base neck stop hurt on h...,n,y,n,n,y
3,like day sound new song,n,y,n,n,y
6,saw nun zombie like also propname tentacleman ...,n,y,n,n,y
7,kentucky miles mile journey home,n,y,n,n,y
8,finish digital paint tablet haywire is contemp...,n,y,n,n,y
9,celebrate new haircut listen swinger music gen...,n,y,n,n,y
10,crush green lantern,n,y,n,n,y
11,magic brain,n,y,n,n,y


**independent and dependent variable split**

In [11]:
# independent and dependent variable split
X = df[['STATUS']]
y = df[["cEXT","cNEU","cAGR","cCON","cOPN"]]

**NLP Columntransformer**

here we are converting text column into Numbers so that machine can understand it.

In [12]:
# Word2Vec Model 
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.models import Word2Vec

class GensimWord2VecVectorizer(BaseEstimator, TransformerMixin):
  
    def __init__(self, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None,
                sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5,
                ns_exponent=0.75, cbow_mean=1, hashfxn=hash, iter=5, null_word=0,
                trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False,
                callbacks=(), max_final_vocab=None):
        self.size = size
        self.alpha = alpha
        self.window = window
        self.min_count = min_count
        self.max_vocab_size = max_vocab_size
        self.sample = sample
        self.seed = seed
        self.workers = workers
        self.min_alpha = min_alpha
        self.sg = sg
        self.hs = hs
        self.negative = negative
        self.ns_exponent = ns_exponent
        self.cbow_mean = cbow_mean
        self.hashfxn = hashfxn
        self.iter = iter
        self.null_word = null_word
        self.trim_rule = trim_rule
        self.sorted_vocab = sorted_vocab
        self.batch_words = batch_words
        self.compute_loss = compute_loss
        self.callbacks = callbacks
        self.max_final_vocab = max_final_vocab

    def fit(self, X, y=None):
        self.model_ = Word2Vec(
            sentences=X, corpus_file=None,
            size=self.size, alpha=self.alpha, window=self.window, min_count=self.min_count,
            max_vocab_size=self.max_vocab_size, sample=self.sample, seed=self.seed,
            workers=self.workers, min_alpha=self.min_alpha, sg=self.sg, hs=self.hs,
            negative=self.negative, ns_exponent=self.ns_exponent, cbow_mean=self.cbow_mean,
            hashfxn=self.hashfxn, iter=self.iter, null_word=self.null_word,
            trim_rule=self.trim_rule, sorted_vocab=self.sorted_vocab, batch_words=self.batch_words,
            compute_loss=self.compute_loss, callbacks=self.callbacks,
            max_final_vocab=self.max_final_vocab)
        return self

    def transform(self, X):
        X_embeddings = np.array([self._get_embedding(words) for words in X])
        return X_embeddings

    def _get_embedding(self, words):
        valid_words = [word for word in words if word in self.model_.wv.vocab]
        if valid_words:
            embedding = np.zeros((len(valid_words), self.size), dtype=np.float32)
            for idx, word in enumerate(valid_words):
                embedding[idx] = self.model_.wv[word]

            return np.mean(embedding, axis=0)
        else:
            return np.zeros(self.size)

In [13]:
from WebApp.app import GensimWord2VecVectorizer
GensimWord2VecVectorizer.__module__ = 'app'

import sys
sys.modules['app'] = sys.modules['WebApp.app']

In [14]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
        ('step1', GensimWord2VecVectorizer(), 'STATUS')
    ],remainder='drop')

**check data for imbalance**

it is very important to check data balance otherwise it will give you unexpected result or bias result in favour of majority class.

In [15]:
unique, counts = np.unique(y["cEXT"], return_counts=True)
print(unique,counts)
unique, counts = np.unique(y["cNEU"], return_counts=True)
print(unique,counts)
unique, counts = np.unique(y["cAGR"], return_counts=True)
print(unique,counts)
unique, counts = np.unique(y["cCON"], return_counts=True)
print(unique,counts)
unique, counts = np.unique(y["cOPN"], return_counts=True)
print(unique,counts)

['n' 'y'] [5338 3983]
['n' 'y'] [5842 3479]
['n' 'y'] [4341 4980]
['n' 'y'] [5041 4280]
['n' 'y'] [2378 6943]


In [16]:
# Balancing the cEXT Class
df_cEXT_y = df[df['cEXT']=='y']
df_cEXT_n = df[df['cEXT']=='n']
print(df_cEXT_y.shape)
print(df_cEXT_n.shape)

df_cEXT_downsampled = df_cEXT_n.sample(df_cEXT_y.shape[0])
print(df_cEXT_downsampled.shape)

df_cEXT_balanced = pd.concat([df_cEXT_downsampled, df_cEXT_y])
print(df_cEXT_balanced.shape)
unique, counts = np.unique(df_cEXT_balanced["cEXT"], return_counts=True)
print(unique,counts)

(3983, 6)
(5338, 6)
(3983, 6)
(7966, 6)
['n' 'y'] [3983 3983]


#cEXT Model

In [17]:
# independent and dependent variable split for cEXT
X_cEXT = df_cEXT_balanced[['STATUS']]
y_cEXT = df_cEXT_balanced[["cEXT"]]

In [18]:
X_cEXT.head()

,STATUS
1143,wonder better idea stay home celebrate hemingw...
9397,aaaaaagggggggggghhhhhhhh hurt bad past days wi...
2534,fb wierd ffs way home page look funny
2748,massive robotinduced coronary day
2575,those owe resume hand them in gon na night ton...


In [19]:
y_cEXT.head()

,cEXT
1143,n
9397,n
2534,n
2748,n
2575,n


In [20]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cEXT,y_cEXT,test_size=0.7)

In [21]:
X_test = pd.concat([X_train,X_test])
y_test = pd.concat([y_train,y_test])

In [22]:
#convert target variable into number
y_train.replace({'y':1, 'n':0}, inplace = True)
y_test.replace({'y':1, 'n':0}, inplace = True)

In [23]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,recall_score, classification_report, cohen_kappa_score
from sklearn import metrics 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import pickle
import numpy as np
import dill
import torch
# ########## 
# SVC - support vector classifier 
# ##########
# defining parameter range
hyperparameters = {'C': [0.1, 1],
              'gamma': [1, 0.1],
              'kernel': ['rbf'],
              'probability':[True]}
model_sv = Pipeline([
        ('column_transformers', ct),
        ('model', GridSearchCV(SVC(), hyperparameters,
                               refit = True, verbose = 3)),
    ])
model_sv_cEXT = model_sv.fit(X_train, y_train['cEXT'])
# Save the trained cEXT - SVM Model.        

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END C=0.1, gamma=1, kernel=rbf, probability=True;, score=0.504 total time=   2.6s
[CV 2/5] END C=0.1, gamma=1, kernel=rbf, probability=True;, score=0.504 total time=   2.7s
[CV 3/5] END C=0.1, gamma=1, kernel=rbf, probability=True;, score=0.504 total time=   2.5s
[CV 4/5] END C=0.1, gamma=1, kernel=rbf, probability=True;, score=0.504 total time=   2.7s
[CV 5/5] END C=0.1, gamma=1, kernel=rbf, probability=True;, score=0.505 total time=   2.5s
[CV 1/5] END C=0.1, gamma=0.1, kernel=rbf, probability=True;, score=0.504 total time=   2.8s
[CV 2/5] END C=0.1, gamma=0.1, kernel=rbf, probability=True;, score=0.504 total time=   2.9s
[CV 3/5] END C=0.1, gamma=0.1, kernel=rbf, probability=True;, score=0.504 total time=   3.3s
[CV 4/5] END C=0.1, gamma=0.1, kernel=rbf, probability=True;, score=0.504 total time=   3.1s
[CV 5/5] END C=0.1, gamma=0.1, kernel=rbf, probability=True;, score=0.505 total time=   3.6s
[CV 1/5] END C=1, ga

In [25]:
import joblib
joblib.dump(model_sv_cEXT, 'model_Word2Vec_sv_cEXT.pkl')

['model_Word2Vec_sv_cEXT.pkl']

In [26]:
model_sv_cEXT.predict(X_test)

array([1, 1, 1, ..., 1, 1, 1])

In [27]:
model_sv_cEXT.predict_proba(X_test)

array([[0.52825251, 0.47174749],
       [0.52823763, 0.47176237],
       [0.5282414 , 0.4717586 ],
       ...,
       [0.52823388, 0.47176612],
       [0.5282419 , 0.4717581 ],
       [0.52826144, 0.47173856]])